In [1]:
import numpy as np
import timeit
import argparse
import cv2
import matplotlib.pyplot as plt
import math
import queue
from Obstacle import *
from Utils.MathUtils import *
from Utils.Node import *
from Utils.Viz import *


In [2]:
def getBranches(node, step_size, goal_state):
    moves = ["1", "2", "3", "4", "5"]
    state = node.getState()
    branches = []
    branches.append(Node(move30Clock(state, step_size), node, moves[0], node.getCost() + step_size))
    branches.append(Node(move30AntiClock(state, step_size), node, moves[1], node.getCost() + step_size))
    branches.append(Node(move60Clock(state, step_size), node, moves[2], node.getCost() + step_size))
    branches.append(Node(move60AntiClock(state, step_size), node, moves[3], node.getCost() + step_size))
    branches.append(Node(moveStraight(state, step_size), node, moves[4], node.getCost() + step_size))

    #remove None nodes
    b = [branch for branch in branches if branch.getState() is not None]
            
    return b

def move30Clock(state, step_size): #assuming we cat land on the borders
  
    current_theta = state[2]
    new_theta = current_theta - 30
    if new_theta <= -360 :
        new_theta = new_theta + 360

    step_x = step_size * np.cos(np.radians(new_theta))
    step_y = step_size * np.sin(np.radians(new_theta))
    
    new_state = [state[0] + step_x, state[1] + step_y, new_theta]

    if isInObstacleSpace(state, new_state):
        return None
 
    return new_state

def move30AntiClock(state, step_size):

    current_theta = state[2]
    new_theta = current_theta + 30
    if new_theta >= 360:
        new_theta = new_theta - 360

    step_x = step_size * np.cos(np.radians(new_theta))
    step_y = step_size * np.sin(np.radians(new_theta))
    
    new_state = [state[0] + step_x, state[1] + step_y, new_theta]

    if isInObstacleSpace(state, new_state):
        return None
 
    return new_state

def move60Clock(state, step_size):

    current_theta = state[2]
    new_theta = current_theta - 60
    if new_theta <= -360:
        new_theta = new_theta + 360
    
    step_x = step_size * np.cos(np.radians(new_theta))
    step_y = step_size * np.sin(np.radians(new_theta))
    
    new_state = [state[0] + step_x, state[1] + step_y, new_theta]
    if isInObstacleSpace(state, new_state):
        return None
 
    return new_state

def move60AntiClock(state, step_size):

    current_theta = state[2]
    new_theta = current_theta + 60
    if new_theta >= 360:
        new_theta = new_theta - 360

    step_x = step_size * np.cos(np.radians(new_theta))
    step_y = step_size * np.sin(np.radians(new_theta))
    
    new_state = [state[0] + step_x, state[1] + step_y, new_theta]

    if isInObstacleSpace(state, new_state):
        return None
 
    return new_state

def moveStraight(state, step_size):

    current_theta = state[2]
    new_theta = current_theta + 0

    step_x = step_size * np.cos(np.radians(new_theta))
    step_y = step_size * np.sin(np.radians(new_theta))
    
    new_state = [state[0] + step_x, state[1] + step_y, new_theta]

    if isInObstacleSpace(state, new_state):
        return None
 
    return new_state



In [3]:
def isInObstacleSpace(parent_state, current_state):

    x1, y1 = parent_state[:2]
    x2, y2 = current_state[:2]
    #move line
    move = getLineParam([x1, y1], [x2, y2])

    if (x2 > 399 or x2 < 0 or y2 < 0 or y2 > 299):
        return True

    if rectIntersect(move) or cIntersect(move) or circleIntersect(move) or ellipseIntersect(move):
        return True
    else:
        return False  

In [4]:
def checkGoalReached(current_node, goal_state, thresh_radius):
    current_state = current_node.getState()
    radius_sq = np.square(current_state[0] - goal_state[0]) + np.square(current_state[1] - goal_state[1])
    if radius_sq < thresh_radius**2:
        return True
    else:
        return False


In [5]:
def checkVisited(node, node_array, goal_state, threshold=0.5, thetaStep=30):
    result = False
    node_state = node.getState()
    x = node_state[0]
    y = node_state[1]
    theta = node_state[2]
    x = int(halfRound(x)/threshold)
    y = int(halfRound(y)/threshold)
    theta = int(theta/thetaStep)

    if (node.getCost() + computeHeuristicCost(node_state, goal_state) < node_array[x, y, theta]):
        result = True
    return result


In [6]:
def computeHeuristicCost(current_state, goal_state):
    cost = 0.0
    if current_state is not None:
        cost =  ((current_state[0]-goal_state[0])**2 + (current_state[1]-goal_state[1])**2)**(0.5)
    return cost

In [7]:
def main():
    h = 300
    w = 400
    space_size = [h, w]
    threshold = 0.5
    step_angle = 30
    step_size = 10
    SaveFileName = "/home/sakshi/courses/ENPM661/proj3_sakshi_kakde/Results/astar.avi"
    visited = np.zeros((int(400/threshold),int(300/threshold),int(360/step_angle)), dtype='int')

    start_point = [20, 50, 90]
    goal_state = [320, 230]

    result = cv2.VideoWriter(SaveFileName,  
                        cv2.VideoWriter_fourcc(*'MJPG'), 
                        300, (sizex, sizey))


    nodes = queue.PriorityQueue()
    init_node = Node(start_point, None, None, 0)

    nodes.put((init_node.getCost(), init_node))

    root2 = np.sqrt(2)

    goal_reached = False
    node_array = np.array([[[math.inf  for k in range(12)] for j in range(int(300/threshold))] for i in range(int(400/threshold))])

    space_size = [300, 400]
    space_map = np.zeros([space_size[0], space_size[1], 3], dtype=np.uint8)
    space_map = updateMap(space_map, init_node, [0,0,255])
    space_map = addObstacles2Map(space_map)

    full_path = None
    goal_reached = False
    while (not nodes.empty()):
        current_node = nodes.get()[1]
        space_map = updateMap(space_map, current_node, [0, 255, 0])
        result.write(space_map)
        # cv2.imshow('frame',space_map)
        # # cv2.waitKey()
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

        if checkGoalReached(current_node, goal_state,5):
            print('Goal reached')
            print("The cost of path: ", current_node.getCost())
            full_path, node_path = current_node.getFullPath()
            goal_reached = True

            for node in node_path:
                space_map = updateMap(space_map, node, [0, 0, 255])
                result.write(space_map)
            #     cv2.imshow('frame',space_map)
            #     if cv2.waitKey(1) & 0xFF == ord('q'):
            #         break
            # cv2.waitKey()
        else:
            branches = getBranches(current_node, step_size, goal_state)    
            for branch_node in branches:

                branch_state = branch_node.getState()
                if checkVisited(branch_node, node_array, goal_state, threshold=0.5, thetaStep=30):
                    node_array[int(halfRound(branch_state[0])/threshold), int(halfRound(branch_state[1])/threshold), int(halfRound(branch_state[2])/30)] = branch_node.getCost() + computeHeuristicCost(branch_state, goal_state)
                    nodes.put((branch_node.getCost() + computeHeuristicCost(branch_state, goal_state), branch_node))

        if (goal_reached): break
            


    cv2.destroyAllWindows()
    

In [8]:
if __name__ == "__main__":
    main()

Goal reached
The cost of path:  460
